<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
# TODO

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
!pip install --upgrade seaborn
import seaborn as sns

sns.__version__

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.9.0)


'0.9.0'

In [3]:
dataurl = "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"

columnnames = ['A'+str(i) for i in range (1, 17)]

df = pd.read_csv(dataurl, header=None, names=columnnames, na_values= '?')

df.isnull().sum()
df.describe()

,A2,A3,A8,A11,A14,A15
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [4]:
# Filling continous data with respective mean

values = {'A2': df.A2.mean(), 'A14': df.A14.mean()}
df.fillna(value=values, inplace=True)

df.describe()

,A2,A3,A8,A11,A14,A15
count,690.000000,690.000000,690.000000,690.00000,690.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.853273,4.978163,3.346513,4.86294,172.159274,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.670000,1.000000,0.165000,0.00000,80.000000,0.000000
50%,28.625000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,37.707500,7.207500,2.625000,3.00000,272.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [5]:
df.describe(exclude=[np.number])

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,678,684,684,681,681,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,468,519,519,137,399,361,395,374,625,383


In [6]:
# Filling categorical data with ffill

df.fillna(method='ffill', inplace=True)

df.describe(exclude=[np.number])

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,690,690,690,690,690,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,476,524,524,139,403,361,395,374,625,383


In [7]:
df.isnull().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

In [8]:
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [9]:
# TODO

dfgrouped = df.groupby('A16')

# Column names with continous data
continousdatacols = ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

for i in continousdatacols:
  print("\nColumn {}\n".format(i))
  print(dfgrouped[i].describe())


Column A2

     count       mean        std    min     25%    50%    75%    max
A16                                                                 
+    307.0  33.706470  12.768888  13.75  23.210  30.67  41.33  76.75
-    383.0  29.854182  10.779084  15.17  22.125  27.67  34.79  80.25

Column A3

     count      mean       std  min    25%   50%   75%     max
A16                                                           
+    307.0  5.904951  5.471485  0.0  1.500  4.46  9.52  28.000
-    383.0  3.839948  4.337662  0.0  0.835  2.21  5.00  26.335

Column A8

     count      mean       std  min    25%    50%  75%     max
A16                                                           
+    307.0  3.427899  4.120792  0.0  0.750  2.000  5.0  28.500
-    383.0  1.257924  2.120481  0.0  0.125  0.415  1.5  13.875

Column A11

     count      mean       std  min  25%  50%  75%   max
A16                                                     
+    307.0  4.605863  6.320242  0.0  0.0  3.0  7.0  67.0


In [0]:
# Spliting the data based on class attribute + and - 

dfA16pos = df[df.A16 == '+']
dfA16pneg = df[df.A16 == '-']

In [11]:
from scipy import stats

s1 = "Features that significantly affects positive class attribute\n"
s2 = "Features that significantly affects negative class attribute\n"
s3 = "Features that have no significant affect on class attribute\n"


for column in continousdatacols:
  statistics, pvalue = stats.ttest_ind(dfA16pos[column], 
                          dfA16pneg[column], 
                          equal_var=True)
  if statistics > 0 and pvalue < 0.01:
    s1 += "{:10} {:<10.5} {:.4}\n".format(column, statistics, pvalue)
  elif statistics < 0 and pvalue < 0.01:
    s2 += "{:10} {:<10.5} {:.4}\n".format(column, statistics, pvalue)
  else:
    s3 += "{:10} {:<10.5} {:.4}\n".format(column, statistics, pvalue)

print("{}\n{}\n{}".format(s1, s2, s3))

Features that significantly affects positive class attribute
A2         4.2959     1.99e-05
A3         5.53       4.552e-08
A8         8.9358     3.671e-18
A11        11.667     7.958e-29
A15        4.6802     3.452e-06

Features that significantly affects negative class attribute
A14        -2.6355    0.008592

Features that have no significant affect on class attribute



In [12]:
# Column names with categorical data
categoricaldatacols = [i for i in df.columns if i not in continousdatacols]
categoricaldatacols.remove('A16')

s1 = "Features that significantly affects class attribute\n"
s2 = "Features that have no significant affect on class attribute\n"

for column in categoricaldatacols:
  statistics, pvalue = stats.chisquare(pd.crosstab(df[column], df.A16), 
                                      axis=None)
  if pvalue < 0.01:
    s1 += "{:10} {:<10.5} {:.4}\n".format(column, statistics, pvalue)
  else:
    s2 += "{:10} {:<10.5} {:.4}\n".format(column, statistics, pvalue)
    
print("{}\n{}".format(s1, s2))

Features that significantly affects class attribute
A1         111.19     6.072e-24
A4         644.66     4.513e-137
A5         644.66     4.513e-137
A6         404.68     5.567e-69
A7         1873.6     0.0
A9         357.83     3.013e-77
A10        173.13     2.684e-37
A12        14.383     0.002428
A13        1037.3     5.125e-222

Features that have no significant affect on class attribute



## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

## t-test results for continous data columns

The output of the t-test results ran for different continous data with respect to class attribute is shared below

Features that significantly affects positive class attribute

| Features|Statistics|pvalue|
|----------|:-------------:|------:|
| A2 |  4.2959 | 1.99e-05 |
| A3 |  5.53 | 4.552e-08 |
|A8|8.9358| 3.671e-18|
|A11|11.667|7.958e-29|
|A15| 4.6802|3.452e-06|

Features that significantly affects negative class attribute

| Features|Statistics|pvalue|
|----------|:-------------:|------:|
| A14 |  -2.6355 | 0.008592 |

#### Based on the t-test output.

* A11 feature seems to have highest significance for credit approval.
* A14 feature seems to have high significance for credit rejection.

## chisquare-test results for categorical data columns

* The output of the chisquare-test results ran for different categorical data with respect to class attribute is shared below

Features that significantly affects class attribute

| Features|Statistics|pvalue|
|----------|:-------------:|------:|
|A1      |   111.19  |   6.072e-24|
|A4      |   644.66  |  4.513e-137|
|A5      |   644.66  |   4.513e-137|
|A6      |   404.68  |   5.567e-69|
|A7      |   1873.6  |   0.0|
|A9      |   357.83  |   3.013e-77|
|A10    |    173.13 |    2.684e-37|
|A12    |    14.383 |    0.002428|
|A13    |    1037.3 |    5.125e-222|

Features that have no significant affect on class attribute

#### Based on the chisquare-test output.

* A7 and A13 feature seems to have higher significance on class attribute as compared to other columns.
* A12 feature seems to have least significance on class attribute as compared to other columns.

## Challenging part of this challenge

* Since the column names were not clear, so relied completely on the results from t-test and chi-square tests help identifying the significance on class attribute.

So the prior knowledge and hypothesis cannot be assumed and hypthesis testing was done accordingly.